In [1]:
!pip install kfp==1.8.22


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import kfp


client = kfp.Client(host="http://localhost:53462")

In [12]:
client.list_experiments()

{'experiments': [{'created_at': datetime.datetime(2023, 9, 9, 10, 52, 34, tzinfo=tzutc()),
                  'description': 'All runs created without specifying an '
                                 'experiment will be grouped here.',
                  'id': '33146035-dd9e-4257-99a4-2e0a199e7923',
                  'name': 'Default',
                  'resource_references': [{'key': {'id': None,
                                                   'type': 'NAMESPACE'},
                                           'name': None,
                                           'relationship': 'OWNER'}],
                  'storage_state': 'STORAGESTATE_AVAILABLE'}],
 'next_page_token': None,
 'total_size': 1}

In [13]:
from kfp.components import create_component_from_func
from kfp.components import InputPath, OutputPath
from typing import NamedTuple

In [5]:
!pip install geopandas geopy osmnx

[notice] A new release of pip available: 22.3.1 -> 23.2.1

[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
def geocode_null_addresses(row):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="POI_app")
    centroid = row["geometry"].centroid
    lat = centroid.y
    lon = centroid.x

    location = geolocator.reverse(f"{lat}, {lon}")

    return location




def get_city_geometry(city: str = "Миасс, Челябинская область"):
    import osmnx as ox

    territory = ox.geocode_to_gdf(city)

    return territory


def get_all_buildings(territory: str = "Миасс, Челябинская область"):
    def get_centroid(row):
        centroid = row["geometry"].centroid

        return centroid
    import osmnx as ox

    buildings = ox.geometries_from_place(territory, {"building": True})
    buildings = buildings.reset_index()

    buildings["centroid"] = buildings.apply(get_centroid, axis=1)

    buildings["lat"] = buildings.centroid.y
    buildings["lon"] = buildings.centroid.x

    buildings = buildings.set_crs(4326)
    cols = ["name", "geometry", "addr:street", "addr:housenumber", "centroid", "lat", "lon"]

    buildings_with_addresses = buildings[~buildings["addr:housenumber"].isna()][cols]
    buildings_without_addresses = buildings[buildings["addr:housenumber"].isna()][cols]

    print(buildings_with_addresses)

    return buildings_with_addresses[["addr:street", "addr:housenumber"]], buildings_without_addresses



def geocode_buildings(buildings):
    buildings["adddress"] = buildings.apply(geocode_null_addresses, axis=1)

    return geocode_buildings





In [15]:
get_city_geometry_step = kfp.components.create_component_from_func(
    func=get_city_geometry,
    output_component_file='get_city_geometry.yaml', # This is optional. It saves the component spec for future use.
    packages_to_install=["osmnx", "geopy", "geopandas"])

In [16]:
get_all_buildings_step = kfp.components.create_component_from_func(
    func=get_all_buildings,
    output_component_file='get_all_buildings.yaml', # This is optional. It saves the component spec for future use.
    packages_to_install=["osmnx", "geopy", "geopandas"])

In [17]:
def get_buildings_data_pipeline():
    get_city_geometry_task = get_city_geometry_step()
    get_all_buildings_task = get_all_buildings_step()
    

# Компилируем пайплайн и сохраняем его в файл pipeline_with_steps.yaml, который дальше можно загрузить через ui для создания полноценного пайплайна
kfp.compiler.Compiler().compile(get_buildings_data_pipeline, 'buildings_data_pipeline.yaml')

In [18]:
client.create_run_from_pipeline_func(get_buildings_data_pipeline, arguments={})

RunPipelineResult(run_id=56fdb60a-18a7-4c9d-9777-6ef84999f8f0)